In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc, script, tx

### Exercise 1




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_parse_segwit`

In [2]:
# Exercise 1

reload(tx)
run(tx.TxTest('test_parse_segwit'))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 2




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_serialize_segwit`

In [3]:
# Exercise 2

reload(tx)
run(tx.TxTest('test_serialize_segwit'))

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


### Exercise 3




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_sig_hash_bip143`

In [4]:
# Exercise 3

reload(tx)
run(tx.TxTest('test_sig_hash_bip143'))

.
----------------------------------------------------------------------
Ran 1 test in 0.124s

OK


In [5]:
# example for creating a bech32 address
from ecc import S256Point
from helper import encode_bech32_checksum, encode_varstr
sec_hex = '039d5ca49670cbe4c3bfa84c96a8c87df086c6ea6a24ba6b809c9de234496808d5'
point = S256Point.parse(bytes.fromhex(sec_hex))
h160 = point.hash160()
raw = bytes([0])
raw += encode_varstr(h160)
bech32 = encode_bech32_checksum(raw, testnet=False)
print(bech32)

bc1qttnpu7attc248hz22jxtyaqkfc7z4qd8yk882v


### Exercise 4
#### Create a testnet bech32 address using your private key from the Session 0

Fill in the spreadsheet with your bech32 address.


In [6]:
# Exercise 4

from ecc import PrivateKey
from helper import encode_bech32_checksum, encode_varstr, hash256, little_endian_to_int
# use the same passphrase from session 0
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
secret = little_endian_to_int(hash256(passphrase))
# create a private key using the secret
private_key = PrivateKey(secret)
# get the public key using the .point property
public_key = private_key.point
# get the hash160 of the point
h160 = public_key.hash160()
# the raw bytes to be encrypted starts with the segwit version (0 or b'\x00)
raw = bytes([0])
# next, add the hash160 using encode_varstr
raw += encode_varstr(h160)
# encode to bech32 using encode_bech32_checksum, remember testnet=True
bech32 = encode_bech32_checksum(raw, testnet=True)
# print the address
print(bech32)

tb1qgqd0pdtu0f9hfyx9pzj86p686q70dtpwkmp0yw


### Exercise 5




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:S256Test:test_bech32_address`

In [7]:
# Exercise 5

reload(ecc)
run(ecc.S256Test('test_bech32_address'))

.
----------------------------------------------------------------------
Ran 1 test in 0.029s

OK


In [8]:
# Example for signing a p2wpkh input
from io import BytesIO
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, SIGHASH_ALL
from tx import Tx
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
raw_tx_hex = '01000000000101cca99b60e1d687e8faaf93e114114e7b5f6382d9f5d45ffb76ac7472ad7d734c0100000000ffffffff014c400f0000000000160014092ab91b37b4182061d9c01199aaac029f89561f0000000000'
input_index = 0
stream = BytesIO(bytes.fromhex(raw_tx_hex))
tx_obj = Tx.parse(stream, testnet=True)
z = tx_obj.sig_hash_bip143(input_index)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
tx_in = tx_obj.tx_ins[input_index]
tx_in.witness = [sig, sec]
print(tx_obj.verify_input(input_index))

True


### Exercise 6




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_sign_p2wpkh`

In [9]:
# Exercise 6

reload(tx)
run(tx.TxTest('test_sign_p2wpkh'))

.
----------------------------------------------------------------------
Ran 1 test in 0.300s

OK


In [10]:
# Example for creating a p2wpkh transaction
from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from script import P2WPKHScriptPubKey
from tx import Tx, TxIn, TxOut
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
prev_tx_hex = '0f007db8670c8b22ed64d95c61895d9c8e516ec938f99fbe4973fc0172ef93cf'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = 1
fee = 500
tx_in = TxIn(prev_tx, prev_index)
amount = tx_in.value(testnet=True) - fee
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
_, _, h160 = decode_bech32(target_address)
script_pubkey = P2WPKHScriptPubKey(h160)
tx_out = TxOut(amount, script_pubkey)
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True, segwit=True)
tx_obj.sign_input(0, private_key)
print(tx_obj.serialize().hex())

01000000000101cf93ef7201fc7349be9ff938c96e518e9c5d89615cd964ed228b0c67b87d000f0100000000ffffffff014c400f00000000001600146e13971913b9aa89659a9f53d327baa8826f2d7502483045022100e606e37820fd935e29955b3d03935beb1fca64922029634f4e1024fbe14bbc950220748ab6bc06054dd422039e9587566d5968faa9f39810d2194a9bcf18b6c092f3012102c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d00000000


### Exercise 7

#### Create a p2wpkh spending transaction

You have been sent 0.05 testnet BTC. Send 0.03 to `tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au`
and the change back to your bech32 address.


In [11]:
# Exercise 7

from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from network import SimpleNode
from script import P2WPKHScriptPubKey
from tx import Tx, TxIn, TxOut
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
private_key = PrivateKey(little_endian_to_int(hash256(passphrase)))
prev_tx_hex = '94448a601fce6961a5fabbc554068460d979b15bee9531e378fbb458bc644378'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = 0
fee = 500
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
target_amount = 3000000
# create the transaction input
tx_in = TxIn(prev_tx, prev_index)
# create an array of tx_outs
tx_outs = []
# decode the target address to get the hash160 of the address
_, _, target_h160 = decode_bech32(target_address)
# create the target script pubkey using P2WPKHScriptPubKey
target_script_pubkey = P2WPKHScriptPubKey(target_h160)
# add the target transaction output
tx_outs.append(TxOut(target_amount, target_script_pubkey))
# calculate the change amount, remember you were sent 5000000 sats
change_amount = 5000000 - target_amount - fee
# calculate the hash160 for your private key
change_h160 = private_key.point.hash160()
# create the change script pubkey using P2WPKHScriptPubKey
change_script_pubkey = P2WPKHScriptPubKey(change_h160)
tx_outs.append(TxOut(change_amount, change_script_pubkey))
# create the transaction with testnet=True and segwit=True
tx_obj = Tx(1, [tx_in], tx_outs, 0, testnet=True, segwit=True)
# sign the one input with your private key
tx_obj.sign_input(0, private_key)
# print the hex to see what it looks like
print(tx_obj.serialize().hex())

01000000000101784364bc58b4fb78e33195ee5bb179d960840654c5bbfaa56169ce1f608a44940000000000ffffffff02c0c62d00000000001600146e13971913b9aa89659a9f53d327baa8826f2d758c821e0000000000160014401af0b57c7a4b7490c508a47d0747d03cf6ac2e02483045022100ace4ba51b732f7098771cf9e6aee2abc03b1c20de01af54dd7e9463f702255d802205911108195c7fc30eab6192cacefa8fe3d4e3b123def84b65ecba399de5851dd012102c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d00000000


### Exercise 8




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_verify_p2wpkh`

In [12]:
# Exercise 8

reload(tx)
run(tx.TxTest('test_verify_p2wpkh'))

.
----------------------------------------------------------------------
Ran 1 test in 0.352s

OK


In [13]:
# Example of generating a p2sh-p2wpkh address
from ecc import S256Point
from helper import encode_base58_checksum, hash160
from script import P2WPKHScriptPubKey, P2SHScriptPubKey
sec_hex = '02c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d'
point = S256Point.parse(bytes.fromhex(sec_hex))
h160 = point.hash160()
redeem_script = P2WPKHScriptPubKey(h160)
p2sh_h160 = hash160(redeem_script.raw_serialize())
p2sh_script_pubkey = P2SHScriptPubKey(p2sh_h160)
address = p2sh_script_pubkey.address(testnet=False)
print(address)

3CobPD6RBnTZsFdka71XHQr4vHXDZMu2zm


### Exercise 9
#### Create a testnet p2sh-p2wpkh address with your private key.



In [14]:
# Exercise 9

from ecc import PrivateKey
from helper import encode_varstr, hash160, hash256, little_endian_to_int
from script import P2WPKHScriptPubKey, P2SHScriptPubKey
# use the same passphrase from session 0
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
secret = little_endian_to_int(hash256(passphrase))
# create a private key using the secret
private_key = PrivateKey(secret)
# get the public key using the .point property
public_key = private_key.point
# get the hash160 of the point
h160 = public_key.hash160()
# create the RedeemScript, which is the P2WPKHScriptPubKey of the hash160
redeem_script = P2WPKHScriptPubKey(h160)
# perform a hash160 on the raw serialization of the RedeemScript
p2sh_h160 = hash160(redeem_script.raw_serialize())
# create a P2SHScriptPubKey using the h160
p2sh_script = P2SHScriptPubKey(p2sh_h160)
# return the address, remember testnet=True
address = p2sh_script.address(testnet=True)
# print the address
print(address)

2N4MoSx2SoExv53GJFEdPuMqL8djPQPH2er


### Exercise 10




#### Make [this test](/edit/session1/script.py) pass: `script.py:TestP2SHScriptPubKey:test_address`

In [15]:
# Exercise 10

reload(script)
run(script.TestP2SHScriptPubKey('test_address'))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 11




#### Make [this test](/edit/session1/ecc.py) pass: `ecc.py:S256Test:test_p2sh_p2wpkh_address`

In [16]:
# Exercise 11

reload(ecc)
run(ecc.S256Test('test_p2sh_p2wpkh_address'))

.
----------------------------------------------------------------------
Ran 1 test in 0.028s

OK


In [17]:
# Example for signing a p2sh-p2wpkh input
from io import BytesIO
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, SIGHASH_ALL
from script import Script
from tx import Tx
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
redeem_script = private_key.point.p2sh_p2wpkh_redeem_script()
raw_tx_hex = '010000000001014e6b786f3cd70ab1ffd75caa6bb252c9888fdca9ca94d40fec24bec3e643d89e0000000000ffffffff014c400f0000000000160014401af0b57c7a4b7490c508a47d0747d03cf6ac2e0000000000'
input_index = 0
stream = BytesIO(bytes.fromhex(raw_tx_hex))
tx_obj = Tx.parse(stream, testnet=True)
z = tx_obj.sig_hash_bip143(input_index, redeem_script=redeem_script)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
tx_in = tx_obj.tx_ins[input_index]
tx_in.witness = [sig, sec]
tx_in.script_sig = Script([redeem_script.raw_serialize()])
print(tx_obj.verify_input(input_index))

True


### Exercise 12




#### Make [this test](/edit/session1/tx.py) pass: `tx.py:TxTest:test_sign_p2sh_p2wpkh`

In [18]:
# Exercise 12

reload(tx)
run(tx.TxTest('test_sign_p2sh_p2wpkh'))

.
----------------------------------------------------------------------
Ran 1 test in 0.325s

OK


In [19]:
# Example for creating a p2sh-p2wpkh transaction
from ecc import PrivateKey
from helper import decode_bech32, hash256, little_endian_to_int
from script import P2WPKHScriptPubKey
from tx import Tx, TxIn, TxOut
private_key = PrivateKey(little_endian_to_int(hash256(b'jimmy@programmingblockchain.com Jimmy Song')))
prev_tx_hex = '6c14a8370da20c7de5ebf216ece3156e99e7d6070442d93b80cdc344b2e80867'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = 1
fee = 500
tx_in = TxIn(prev_tx, prev_index)
amount = tx_in.value(testnet=True) - fee
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
_, _, h160 = decode_bech32(target_address)
script_pubkey = P2WPKHScriptPubKey(h160)
tx_out = TxOut(amount, script_pubkey)
tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True, segwit=True)
redeem_script = private_key.point.p2sh_p2wpkh_redeem_script()
tx_obj.sign_input(0, private_key, redeem_script=redeem_script)
print(tx_obj.serialize().hex())

010000000001016708e8b244c3cd803bd9420407d6e7996e15e3ec16f2ebe57d0ca20d37a8146c0100000017160014401af0b57c7a4b7490c508a47d0747d03cf6ac2effffffff0198801e00000000001600146e13971913b9aa89659a9f53d327baa8826f2d7502483045022100e92c1dc1066c1614da514d0e4e97feb8aec96d88598d9b2f9e7840948dfd353f02201fe146c5ec7af8a9a22faed1726f527a6d1c4cf2243476f10637241e2f98f7af012102c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d00000000


### Exercise 13

#### Create a p2sh-p2wpkh spending transaction

You have been sent 0.05 testnet BTC. Send 0.03 to `tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au`
and the change back to your p2sh-p2wpkh address.


In [20]:
# Exercise 13

from ecc import PrivateKey
from helper import decode_bech32, decode_base58, hash256, little_endian_to_int
from network import SimpleNode
from script import P2WPKHScriptPubKey, P2SHScriptPubKey
from tx import Tx, TxIn, TxOut
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
private_key = PrivateKey(little_endian_to_int(hash256(passphrase)))
prev_tx_hex = '9b3e6e253c79672a4e60a8a563ee735c8f87351058a7a0f5d53a2b5771cf6a6d'
prev_tx = bytes.fromhex(prev_tx_hex)
prev_index = 0
fee = 500
target_address = 'tb1qdcfewxgnhx4gjev6nafaxfa64zpx7tt470r3au'
target_amount = 3000000
# create the transaction input
tx_in = TxIn(prev_tx, prev_index)
# create an array of tx_outs
tx_outs = []
# decode the target address to get the hash160 of the address
_, _, target_h160 = decode_bech32(target_address)
# create the target script pubkey using P2WPKHScriptPubKey
target_script_pubkey = P2WPKHScriptPubKey(target_h160)
# add the target transaction output
tx_outs.append(TxOut(target_amount, target_script_pubkey))
# calculate the change amount, remember you were sent 5000000 sats
change_amount = 5000000 - target_amount - fee
# get the p2sh-p2wpkh address for using your private key
p2sh_address = private_key.point.p2sh_p2wpkh_address()
# get the hash160 by decoding the p2sh-p2wpkh address
change_h160 = decode_base58(p2sh_address)
# create the change script pubkey using P2SHScriptPubKey
change_script_pubkey = P2SHScriptPubKey(change_h160)
tx_outs.append(TxOut(change_amount, change_script_pubkey))
# create the transaction with testnet=True and segwit=True
tx_obj = Tx(1, [tx_in], tx_outs, 0, testnet=True, segwit=True)
# grab the RedeemScript from the public point
redeem_script = private_key.point.p2sh_p2wpkh_redeem_script()
# sign the one input with your private key
tx_obj.sign_input(0, private_key, redeem_script=redeem_script)
# print the hex to see what it looks like
print(tx_obj.serialize().hex())

010000000001016d6acf71572b3ad5f5a0a7581035878f5c73ee63a5a8604e2a67793c256e3e9b0000000017160014401af0b57c7a4b7490c508a47d0747d03cf6ac2effffffff02c0c62d00000000001600146e13971913b9aa89659a9f53d327baa8826f2d758c821e000000000017a91479e7cf6859a7047b099a078a8ffbbb58b73b8633870247304402205e35a2329e08e949d16189b213586feee2df37a48f961808372bbee44fcbc59402207a66acbd49f1366ff8e5a718d9f4d33867fe4c595b4834822fb6b33e7e99ab98012102c3700ce19990bccbfa1e072d287049d7c0e07ed15c9aeac84bbc2c38ea667a5d00000000
